# 测试streamlit环境下流式输出文本缺第一个字符的问题

问题结论是llama_index库版本问题。

In [1]:
%%time

import logging
import sys

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import StorageContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from IPython.display import Markdown, display
from llama_index.core import Settings
from llama_index.embeddings.ollama import OllamaEmbedding

from llama_index.llms.openai_like import OpenAILike

CPU times: user 2.66 s, sys: 564 ms, total: 3.23 s
Wall time: 3.88 s


In [2]:
Settings.chunk_size=128
Settings.chunk_overlap=10

Settings

_Settings(_llm=None, _embed_model=None, _callback_manager=None, _tokenizer=None, _node_parser=SentenceSplitter(include_metadata=True, include_prev_next_rel=True, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7f557edfc910>, id_func=<function default_id_func at 0x7f563d11b010>, chunk_size=128, chunk_overlap=10, separator=' ', paragraph_separator='\n\n\n', secondary_chunking_regex='[^,.;。？！]+[,.;。？！]?'), _prompt_helper=None, _transformations=None)

In [3]:
%%time

llm = OpenAILike(model="xiaoyu", 
                 api_base="http://192.168.0.72:3000/v1", 
                 api_key="sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750",
                 is_chat_model=True,
                 temperature=0.1
                )

Settings.llm =llm

CPU times: user 117 ms, sys: 7.49 ms, total: 124 ms
Wall time: 142 ms


In [4]:
%%time

# 初始化全局 embedding 模型
from llama_index.embeddings.ollama import OllamaEmbedding

ollama_embedding = OllamaEmbedding(
    model_name="dztech/bge-large-zh:v1.5",
    # model_name="bge-m3:latest",
    base_url="http://192.168.0.72:11435",
    ollama_additional_kwargs={"mirostat": 0}, # -mirostat N 使用 Mirostat 采样。
)

Settings.embed_model = ollama_embedding

CPU times: user 459 ms, sys: 31.2 ms, total: 490 ms
Wall time: 586 ms


In [5]:
%%time

from milvus import default_server
from pymilvus import connections, utility

default_server.set_base_dir('milvus_data')

# (OPTIONAL) if you want cleanup previous data
default_server.cleanup()

# Start your milvus server
default_server.start()

# Now you could connect with localhost and the given port
# Port is defined by default_server.listen_port
connections.connect(host='127.0.0.1', port=default_server.listen_port)

# Check if the server is ready.
print(utility.get_server_version())

v2.3.5-lite
CPU times: user 3.56 s, sys: 296 ms, total: 3.85 s
Wall time: 7.51 s


In [6]:
%%time

# load documents
documents = SimpleDirectoryReader(input_files=["./books1/故乡.txt"]).load_data()
print("Document ID:", documents[0].doc_id)

Document ID: dacc8ca0-c7c1-46f8-b97c-f0e4accb3361
CPU times: user 4.95 ms, sys: 7.08 ms, total: 12 ms
Wall time: 18.4 ms


In [7]:
%%time

from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.vector_stores.milvus import MilvusVectorStore

vector_store = MilvusVectorStore(dim=1024, overwrite=True)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)

CPU times: user 1.08 s, sys: 98.3 ms, total: 1.17 s
Wall time: 13 s


In [8]:
%%time

# Query Data
query_engine = index.as_query_engine(
    streaming=True,
    similarity_top_k=100,
    similarity_cutoff=0.5
)

CPU times: user 191 ms, sys: 0 ns, total: 191 ms
Wall time: 203 ms


In [9]:
%%time

streaming_response = query_engine.query("闰土是谁")
streaming_response.print_response_stream()
print()

闰土是《故乡》中鲁迅笔下的一位重要角色，一个十一二岁的少年，以项带银圈的形象出现，象征着他的童年记忆和父亲的关爱。他在小说中代表了农村生活的纯真与活力，但随着故事的发展，他经历了生活的艰辛和社会变迁，性格上发生了变化，与“我”产生了隔阂。闰土是那个时代农村少年的缩影，反映了社会的变迁和农民的生活状态。
CPU times: user 431 ms, sys: 15.4 ms, total: 446 ms
Wall time: 22.8 s


In [10]:
%%time

streaming_response = query_engine.query("闰土是谁")
for token in streaming_response.response_gen:
    print(token, end="")
print()

闰土是鲁迅在《呐喊》这部作品中描绘的一个重要角色，他是小说中主人公鲁迅在少年时期的朋友，来自浙江沿海的农村，以他的海边劳作和对猹的勇敢形象为人们所熟知。他曾经戴着银圈，这在文中被提及是他的标志之一。闰土的故事反映了那个时代的社会变迁和人物命运的痕迹。
CPU times: user 258 ms, sys: 21 ms, total: 279 ms
Wall time: 20.6 s


In [11]:
%%time

# Query Data
query_engine = index.as_query_engine(
    streaming=True,
    similarity_top_k=3,
    similarity_cutoff=0.5
)

CPU times: user 186 µs, sys: 0 ns, total: 186 µs
Wall time: 190 µs


In [13]:
%%time

streaming_response = query_engine.query("孔乙己是谁")
for token in streaming_response.response_gen:
    print(token, end="")
print()
print(len(streaming_response.source_nodes))
streaming_response.source_nodes

孔乙己是鲁迅先生笔下《呐喊》中的人物，他是一个穷困潦倒的读书人，以摆弄茴香豆和喝酒为生，常被人取笑和嘲笑。与提供的文本内容无关。
3
CPU times: user 48.1 ms, sys: 4.61 ms, total: 52.8 ms
Wall time: 2.38 s


[NodeWithScore(node=TextNode(id_='84d6a401-4a01-452f-b9ac-842bb138f9a3', embedding=None, metadata={'file_path': 'books1/故乡.txt', 'file_name': '故乡.txt', 'file_type': 'text/plain', 'file_size': 17811, 'creation_date': '2024-05-10', 'last_modified_date': '2024-05-10'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='dacc8ca0-c7c1-46f8-b97c-f0e4accb3361', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_path': 'books1/故乡.txt', 'file_name': '故乡.txt', 'file_type': 'text/plain', 'file_size': 17811, 'creation_date': '2024-05-10', 'last_modified_date': '2024-05-10'}, hash='4857463c4623b1869a7c4c5866f1b8f803834c1bd4b6cd1cc82808263dab39af'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='41ae9985-1052